<a href="https://colab.research.google.com/github/arriyan-aa/AI-Hand-Computer-Vision/blob/main/cv_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1> <b>AI-Hand-Computer-Vision </b> </center> </h1>

Distinguishing between AI-generated hands vs. real hands. Our project aims to address the issue of AI in art and other media. This is an important problem because the use of AI generated material in art and media is becoming increasingly more common and has brought up discussions of ethics and copyright. This model would help distinguish real human hands versus AI generated ones.

# Libraries and Getting Files

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas
!pip install matplotlib
!pip install numpy

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# Dataset Cleaning

Making sure that we are not missing any files and that the naming conventions are all uniform

In [ ]:
# Set the path to the dataset
dataset_path = "drive/MyDrive/ComputerVisionData/hand-data"

# Get the list of all images paths in the dataset
image_paths = [os.path.join(dataset_path, "images", image_name) for image_name in os.listdir(os.path.join(dataset_path, "images"))]
# Get the list of all labels paths in the dataset
label_paths = [os.path.join(dataset_path, "labels", label_name) for label_name in os.listdir(os.path.join(dataset_path, "labels"))]

print (image_paths)

In [ ]:
#we already have a list of images' paths and labels' path, stored in variables : image_paths and label_paths
print(image_paths)
print(label_paths)

Now, we make a list of only the names of images' files and labels' files.

e.g. only 000000000531 without the extension

In [ ]:
# Get the list of filenames without extensions
image_files = {file.split("/")[-1].split(".")[0] for file in image_paths}
label_files = {file.split("/")[-1].split(".")[0] for file in label_paths}

In [ ]:
# Find extra files in each folder
extra_images = image_files - label_files
extra_labels = label_files - image_files

# Output the results
print(f"Extra images (without corresponding labels): {extra_images}")
print(f"Extra labels (without corresponding images): {extra_labels}")

In [ ]:
for file in extra_images:
     os.remove(os.path.join(dataset_path,"images", file + '.jpg')) # or '.png' depending on our image format

for file in extra_labels:
     os.remove(os.path.join(dataset_path,"labels", file + '.txt'))


Now removing them from the dataset:

In [ ]:
os.path.join(dataset_path,"images", file + '.jpg')

Check again if it worked:

In [ ]:
# Get the list of all images paths in the dataset
image_paths = [os.path.join(dataset_path, "images", image_name) for image_name in os.listdir(os.path.join(dataset_path, "images"))]

# Get the list of all labels paths in the dataset
label_paths = [os.path.join(dataset_path, "labels", label_name) for label_name in os.listdir(os.path.join(dataset_path, "labels"))]

# Get the list of filenames without extensions
image_files = {file.split("/")[-1].split(".")[0] for file in image_paths}
label_files = {file.split("/")[-1].split(".")[0] for file in label_paths}
# Find extra files in each folder
extra_images = image_files - label_files
extra_labels = label_files - image_files

# Output the results
print(f"Extra images (without corresponding labels): {extra_images}")
print(f"Extra labels (without corresponding images): {extra_labels}")

# Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/requirements.txt) and check software and hardware.

In [3]:
!pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.1/107.7 GB disk)


YOLOv8 may be used directly in the Command Line Interface (CLI) with a `yolo` command for a variety of tasks and modes and accepts additional arguments, i.e. `imgsz=640`. See a full list of available `yolo` [arguments](https://docs.ultralytics.com/usage/cfg/) and other details in the [YOLOv8 Predict Docs](https://docs.ultralytics.com/modes/train/).


if CLI the format should be:

    yolo TASK MODE ARGS

  Where:

    TASK (optional) is one of (detect, segment, classify, pose)


    MODE (required) is one of (train, val, predict, export, track)


    ARGS (optional) are arg=value pairs like imgsz=640 that override defaults.


Default ARG values are defined on this page from the cfg/defaults.yaml file.


In [4]:
%matplotlib inline
from PIL import Image

from ultralytics import YOLO
model = YOLO("yolo11n-cls.pt")

# Training

Train YOLOv8 on Detect, Segment, Classify and Pose datasets. See YOLOv8 Train Docs for more information.

In [5]:
#@title Select YOLOv8 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir .

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.3 MB/s eta 0:00:00


COMET WARNING: comet_ml.init() is deprecated and will be removed soon. Please use comet_ml.login()


In YOLO (You Only Look Once), a YAML file is used for configuration and setup. It specifies parameters such as paths to datasets, model architecture, training hyperparameters, and class names. The YAML file is essential for defining how the YOLO model should be trained and what it should detect, making it an integral part of customizing the YOLO model for specific object detection tasks.

In [6]:
# Load YOLOv11n-cls, train it on hand dataset for 3 epochs
from ultralytics import YOLO

model = YOLO('yolo11n-cls.pt')  # load a pretrained YOLOv11n-cls classification model
model.train(data='drive/MyDrive/ComputerVisionData/hand-data', epochs=10,imgsz=416)  # train the model

# PARTNER 2 TODO:
# Test different epoch values [between 20-30 is a good number]
# Look at results, maybe take some rough notes abt what its showing (accuracy, confusions), and tweak dataset to improve

# ***
# [keep in mind]
# An epoch = one full pass over the entire dataset
# More epochs = more learning time, but risk of overfitting (the model memorizes the data instead of generalizing)
# Since we are finetuning a pre-trained model, 20-30 epochs could be enough to converge without overfitting
# ***

# ***
# [a note abt imgsz]
# imgsz = size of image
# The larger the number, the more details in the image it considers (ex. small features, textures, shadows)
# A larger value will be slower/take up more GPU memory, so try testing lower values if needed (ex. imgsz= 320 or imgsz=416)
# ***




Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=drive/MyDrive/ComputerVisionData/hand-data, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/arriyan/general/b7a7dbe355474886aaa315decfdedc6b



Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      


COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  9                  -1  1    249728  ultralytics.nn.modules.block.C2PSA           [256, 256, 1]                 
 10                  -1  1    332802  ultralytics.nn.modules.head.Classify         [256, 2]                      
YOLO11n-cls summary: 86 layers, 1,533,666 parameters, 1,533,666 gradients, 3.3 GFLOPs
Tr

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : chief_asset_6838
COMET INFO:     url                   : https://www.comet.com/arriyan/general/b7a7dbe355474886aaa315decfdedc6b
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [11]                : (0.00018170299999999996, 0.0013309920711111112)
COMET INFO:     lr/pg1 [11]                : (0.00018170299999999996, 0.0013309920711111112)
COMET INFO:     lr/pg2 [11]                : (0.00018170299999999996, 0.0013309920711111112)
COMET INFO:     metrics/accuracy_top1 [11] : (0.42, 0.76)
COMET INFO:     metrics/accuracy_top5      : 1.0
COMET INFO:     model/GFLOPs               : 3.254
COMET INFO:     model/parameters           : 153366

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a9a413029f0>
curves: []
curves_results: []
fitness: 0.8799999952316284
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.7599999904632568, 'metrics/accuracy_top5': 1.0, 'fitness': 0.8799999952316284}
save_dir: PosixPath('/content/runs/classify/train')
speed: {'preprocess': 0.0014480199964358083, 'inference': 55.70587264000096, 'loss': 5.6873335173198335e-05, 'postprocess': 0.00013380666738764072}
task: 'classify'
top1: 0.7599999904632568
top5: 1.0

In [7]:
model= YOLO("runs/classify/train/weights/best.pt")
metrics = model.val()  # no arguments needed, dataset and settings remembered
print(metrics)

Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
YOLO11n-cls summary (fused): 47 layers, 1,528,586 parameters, 0 gradients, 3.2 GFLOPs
train: /content/drive/MyDrive/ComputerVisionData/hand-data/train... found 1200 images in 2 classes ✅ 
val: /content/drive/MyDrive/ComputerVisionData/hand-data/val... found 150 images in 2 classes ✅ 
test: /content/drive/MyDrive/ComputerVisionData/hand-data/test... found 150 images in 2 classes ✅ 
val: Fast image access ✅ (ping: 0.8±0.5 ms, read: 171.1±77.7 MB/s, size: 1295.7 KB)
val: Scanning /content/drive/MyDrive/ComputerVisionData/hand-data/val... 150 images, 0 corrupt: 100% ━━━━━━━━━━━━ 150/150 128.2it/s 1.2s
val: New cache created: /content/drive/MyDrive/ComputerVisionData/hand-data/val.cache
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 10/10 0.3it/s 34.7s
                   all       0.76          1
Speed: 0.0ms preprocess, 45.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results sav

In [11]:
# running inference with yolo model on single image
results=model('drive/MyDrive/ComputerVisionData/hand-data/test/ai/72.jpg', save=True)
#results
for r in results:
    print(r.probs)  # print the Probs object containing the detected class probabilities


image 1/1 /content/drive/MyDrive/ComputerVisionData/hand-data/test/ai/72.jpg: 416x416 ai 1.00, real 0.00, 45.7ms
Speed: 12.4ms preprocess, 45.7ms inference, 0.1ms postprocess per image at shape (1, 3, 416, 416)
Results saved to /content/runs/classify/predict
ultralytics.engine.results.Probs object with attributes:

data: tensor([9.9960e-01, 4.0034e-04])
orig_shape: None
shape: torch.Size([2])
top1: 0
top1conf: tensor(0.9996)
top5: [0, 1]
top5conf: tensor([9.9960e-01, 4.0034e-04])


# Downloading Files
Downloading as a zip file onto Google Drive

In [14]:
import shutil
shutil.make_archive('runs', 'zip', 'runs')

'/content/runs.zip'